# Análise de demonstrativos financeiros
## Indicadores de Liquidez

## 1. Carrega os pacotes

In [11]:
# Carrega as bibliotecas
!pip install wget
import wget # biblioteca para realizar o download de sites
import pandas as pd # biblioteca para manipulação de dados e data frames
from zipfile import ZipFile # biblioteca para extrair arquivo zipados

## 2. Cria função para coletar demonstrativos da CVM

In [12]:
def download_extract_concatenate_dfp_files(start_year, end_year, financial_statements):
    """
    Função para realizar o download, extração e concatenação de arquivos zipados contendo os dados da Demonstração Financeira
    Padronizada (DFP) de companhias abertas disponibilizados pela Comissão de Valores Mobiliários (CVM).

    Parâmetros:
    start_year (int): Ano inicial da coleta dos dados.
    end_year (int): Ano final da coleta dos dados.
    financial_statements (list): Lista de nomes dos arquivos de demonstrativos financeiros que deseja-se concatenar. Consolidado: BPA_con (balanço patrimonial ativo);
     BPP_con (Balanço patrimonial passivo) DRE_con; DFC_MI_con; DFC_MD_CON; DMPL_con.


    Retorna:
    Arquivo .csv no diretório principal
    """
    url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"
    # Cria uma lista vazia para inserir os nomes dos arquivos zipados
    arquivo_zipado = []

    # Define os nomes dos arquivos zipados de acordo com o range de datas
    for ano in range(start_year, end_year + 1):
        arquivo_zipado.append(f'dfp_cia_aberta_{ano}.zip')

    # Realiza o download dos arquivos zipados de acordo com a url base
    for arquivos in arquivo_zipado:
        wget.download(url + arquivos)

    # Extrai os arquivos zipados
    for arquivos in arquivo_zipado:
        ZipFile(arquivos, 'r').extractall('DFP')

    # Concatena os dados dos demonstrativos financeiros em um único DataFrame
    for demons in financial_statements:
        arquivo_demonstrativo = pd.DataFrame()
        for ano in range(start_year, end_year + 1):
            arquivo_demonstrativo = pd.concat([arquivo_demonstrativo, pd.read_csv(f'DFP/dfp_cia_aberta_{demons}_{ano}.csv', sep = ';', decimal = ',', encoding = 'ISO-8859-1')])
        arquivo_demonstrativo.to_csv(f'dfp_cia_aberta_{demons}_{start_year}-{end_year}.csv', index = False)

## 3. Coleta os dados

In [13]:
# Coleta os dados
download_extract_concatenate_dfp_files(2010, 2023, ['BPA_con', 'BPP_con'])

In [14]:
# Realiza a leitura do arquivo
bpa = pd.read_csv('/content/dfp_cia_aberta_BPA_con_2010-2022.csv')

In [15]:
# Realiza a leitura do arquivo
bpp = pd.read_csv('/content/dfp_cia_aberta_BPP_con_2010-2022.csv')

## 4. Tratamento de dados

In [16]:
# Empilha os dados
dados = pd.concat([bpa, bpp], axis = 0)

# Realiza o filtro das contas contábeis, do código CVM e da ordem do exercício
dados = dados[dados.CD_CONTA.isin(["1.01.01", # Caixa
                          "2.01",    # Passivo Circulante
                          "1.01",    # Ativo Circulante
                          "1.01.04", # Estoques
                          "1.01.07", # Despesas Antecipadas
                          "1.02",    # Ativo não circulante
                          "2.02" # Passivo não circulante
                          ]) & (dados.CD_CVM == 14311) & (dados.ORDEM_EXERC == 'ÚLTIMO')]
# Seleciona as colunas
dados = dados[["DT_REFER", "CD_CONTA", "DS_CONTA", "VL_CONTA"]]

In [17]:
dados

,DT_REFER,CD_CONTA,DS_CONTA,VL_CONTA
39199,2010-12-31,1.01,Ativo Circulante,4157790.0
39201,2010-12-31,1.01.01,Caixa e Equivalentes de Caixa,1794416.0
39233,2010-12-31,1.01.04,Estoques,121424.0
39245,2010-12-31,1.01.07,Despesas Antecipadas,4965.0
39255,2010-12-31,1.02,Ativo Não Circulante,13701642.0
...,...,...,...,...
854739,2020-12-31,2.02,Passivo Não Circulante,16879754.0
958508,2021-12-31,2.01,Passivo Circulante,7979993.0
958606,2021-12-31,2.02,Passivo Não Circulante,19382307.0
1067002,2022-12-31,2.01,Passivo Circulante,7156597.0


In [18]:
# Converte a coluna de data para datetime
dados['DT_REFER'] = pd.to_datetime(dados['DT_REFER'])

# Cria a coluna Ano
dados['Ano'] = dados['DT_REFER'].dt.year

dados_pivot = dados.pivot(index = 'Ano', columns = 'DS_CONTA', values = 'VL_CONTA')

In [19]:
dados_pivot

DS_CONTA,Ativo Circulante,Ativo Não Circulante,Caixa e Equivalentes de Caixa,Despesas Antecipadas,Estoques,Passivo Circulante,Passivo Não Circulante
Ano,,,,,,,
2010,4157790.0,13701642.0,1794416.0,4965.0,121424.0,2536801.0,4026805.0
2011,3702013.0,15419650.0,1049125.0,4588.0,103802.0,2058821.0,4993314.0
2012,4699255.0,16512299.0,1483137.0,5004.0,124809.0,2847818.0,5866238.0
2013,4680284.0,18431161.0,1741632.0,19982.0,139278.0,3347885.0,6834808.0
2014,5218176.0,20399966.0,740131.0,20133.0,150622.0,4055393.0,7879969.0
2015,6933397.0,22014260.0,1480727.0,49282.0,131018.0,4789118.0,9574061.0
2016,4402990.0,26031219.0,982073.0,39096.0,130637.0,5656036.0,9622727.0
2017,5701834.0,27460543.0,1040075.0,39867.0,110559.0,6109914.0,11541960.0
2018,6677846.0,29252254.0,1948409.0,40819.0,116285.0,6695114.0,12898772.0


## 5. Cálculo dos Indicadores de Liquidez

In [20]:
(
  dados_pivot
 .assign(liquidez_imediata = dados_pivot['Caixa e Equivalentes de Caixa'] / dados_pivot['Passivo Circulante'],
         liquidez_seca = (dados_pivot['Ativo Circulante']  -  dados_pivot['Estoques'] - dados_pivot['Despesas Antecipadas']) / dados_pivot['Passivo Circulante'],
         liquidez_corrente = dados_pivot['Ativo Circulante'] / dados_pivot['Passivo Circulante'],
         liquidez_geral = (dados_pivot['Ativo Circulante'] + dados_pivot['Ativo Não Circulante']) / (dados_pivot['Passivo Circulante'] + dados_pivot['Passivo Não Circulante'])
       )
  )

DS_CONTA,Ativo Circulante,Ativo Não Circulante,Caixa e Equivalentes de Caixa,Despesas Antecipadas,Estoques,Passivo Circulante,Passivo Não Circulante,liquidez_imediata,liquidez_seca,liquidez_corrente,liquidez_geral
Ano,,,,,,,,,,,
2010,4157790.0,13701642.0,1794416.0,4965.0,121424.0,2536801.0,4026805.0,0.707354,1.589167,1.638989,2.720979
2011,3702013.0,15419650.0,1049125.0,4588.0,103802.0,2058821.0,4993314.0,0.509576,1.745476,1.798123,2.711471
2012,4699255.0,16512299.0,1483137.0,5004.0,124809.0,2847818.0,5866238.0,0.520798,1.604541,1.650125,2.434177
2013,4680284.0,18431161.0,1741632.0,19982.0,139278.0,3347885.0,6834808.0,0.520219,1.350412,1.397982,2.269679
2014,5218176.0,20399966.0,740131.0,20133.0,150622.0,4055393.0,7879969.0,0.182505,1.244619,1.286725,2.146407
2015,6933397.0,22014260.0,1480727.0,49282.0,131018.0,4789118.0,9574061.0,0.309186,1.410092,1.447740,2.015407
2016,4402990.0,26031219.0,982073.0,39096.0,130637.0,5656036.0,9622727.0,0.173633,0.748449,0.778459,1.991929
2017,5701834.0,27460543.0,1040075.0,39867.0,110559.0,6109914.0,11541960.0,0.170227,0.908590,0.933210,1.878689
2018,6677846.0,29252254.0,1948409.0,40819.0,116285.0,6695114.0,12898772.0,0.291020,0.973955,0.997421,1.833740
